In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime as dt
import numpy as np
import matplotlib.dates as mdates
import plotly.graph_objects as go
plt.style.use('fivethirtyeight')
colors=plt.rcParams['axes.prop_cycle'].by_key()['color']
%matplotlib widget
MEAN_RADIUS=6378.137
METERS_TO_FEET=3.2808399

In [2]:
paths=os.scandir('gps_data')
path = [x.path for x in paths if 'July21' in x.name][0]
df=pd.read_csv(path)
df=df.dropna(how='all',axis=1).drop(columns=['ID','trksegID'])
df.time=pd.to_datetime(df.time,utc=False)-dt.timedelta(hours=7) # matplotlib doesn't like a TZ aware timestamp
df.time=df.time.dt.tz_localize(None)
df['ele_ft']=df.ele*METERS_TO_FEET
df=df.set_index('time')
df=df.resample('30s').mean().reset_index()
df=df.dropna(subset=['lat','lon','ele'])
df['date']=df.time.dt.strftime('%D')

In [3]:
df['seconds']=(df.time-df.time.min()).dt.total_seconds()
dlat=df.lat.diff()*np.pi/180
dlon=df.lon.diff()*np.pi/180
a=(np.sin(dlat/2))**2+np.cos(df.lat.shift(1)*np.pi/180)*np.cos(df.lat*np.pi/180)*(np.sin(dlon/2))**2
c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
df['dist']=MEAN_RADIUS*c*1000*METERS_TO_FEET
df['dist_total']=df.dist.cumsum()/5280

df['dt']=df.seconds.diff()
df['vel']=(df.dist/df.dt)*3600/5280
df['vel']=df.rolling('600s',on='time').median().vel
df['ele_ft_cum']=df.ele_ft.diff().where(df.ele_ft.diff()>0,other=0).cumsum()

In [4]:
fig,ax=plt.subplots(4,1,figsize=(16,12),sharex=True)
fig.suptitle('Elevation and Speed Plots for July 2021 Trip',weight='bold')
locator = mdates.AutoDateLocator(minticks=7, maxticks=10)
formatter = mdates.ConciseDateFormatter(locator)
for ax_ in ax:
    ax_.xaxis.set_major_locator(locator)
    ax_.xaxis.set_major_formatter(formatter)

ax[0].plot(df.time,df.dist_total,marker='.',linewidth=1,color=colors[2])
ax[1].plot(df.time,df.ele_ft,marker='.',linewidth=1,color=colors[1])
ax[2].plot(df.time,df.vel,marker='.',linewidth=1,color=colors[0])
ax[3].plot(df.time,df.ele_ft_cum,marker='.',linewidth=1,color=colors[3])

ax[0].set_ylabel('Distance [mi]',labelpad=25)
ax[1].set_ylabel('Elevation [ft]')
ax[2].set_ylabel('Speed [ft/s]',labelpad=30)
ax[3].set_ylabel('Elevation Gain [ft]')

titles=['Distance Traveled','Elevation Above Sea Level','Speed','Cummulative Elevation Gain']
i=0
for ax_ in ax:
    ax_.set_title(titles[i],loc='right')
    i+=1

# ax[-1].set_xlim(left=df.time.min(),right=df.time.max())
# ax[-1].set_xlabel('Date')


FigureCanvasNbAgg()

In [5]:
fig = go.Figure()

tracks=[]

for date in df.date.unique():
    mask=df.date==date
    tracks.append(go.Scattermapbox(
                    mode = "markers+lines",
                    lon = df[mask].lon,
                    lat = df[mask].lat,
                    marker = {'size': 10},
                    legendgroup=date,
                    name=date,
                    hovertext=df[mask].ele_ft.astype(int).astype(str) + ' ft, ' + (df[mask].dist_total-df[mask].dist_total.min()).round(1).astype(str) + ' mi ',
                    hoverinfo='text')
    )

for track in tracks:
    fig.add_trace(track)

# fig.update_layout(
#     margin ={'l':0,'t':0,'b':0,'r':0},
#     mapbox = {
#         'center': {'lon': df.lon.mean(), 'lat': df.lat.mean()},
#         'style': 'stamen-terrain',
#         'zoom': 10},
#     width=1600, height=900)

mapbox_layers=[{"below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": "United States Geological Survey",
                "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]}]

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': df.lon.mean(), 'lat': df.lat.mean()},
        'style': 'white-bg',
        'zoom': 10,
        'layers':mapbox_layers},
    width=1600, height=900,
    legend={'yanchor':'top','xanchor':'left','x':0.01,'y':0.99,'bgcolor':'dimgray','font':dict(color='white')})

fig.show()